In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población media anual,513170
1,Huelva,Confirmados PDIA,72844
2,Huelva,Confirmados PDIA 14 días,6462
3,Huelva,Tasa PDIA 14 días,"1259,231833505466"
4,Huelva,Confirmados PDIA 7 días,2416
5,Huelva,Tasa PDIA 7 dias,"470,7991503790167"
6,Huelva,Total Confirmados,73088
7,Huelva,Curados,56575
8,Huelva,Fallecidos,442


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  72844.0


/tmp/ipykernel_87634/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  21247.0


/tmp/ipykernel_87634/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]

/tmp/ipykernel_87634/242626003.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_87634/242626003.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_87634/242626003.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población media anual']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 212 personas en los últimos 7 días 

Un positivo PCR cada 78 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población media anual' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Rename columns
tabla.columns = ['Confirmados PDIA', 'Confirmados PDIA 14 días', 'Confirmados PDIA 7 días', 'Población']

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,72844.0,6462.0,2416.0,513170.0,470.799150,1259.231834,792.0
Huelva-Costa,42049.0,3201.0,1150.0,289548.0,397.170763,1105.516184,330.0
Sierra de Huelva-Andévalo Central,9680.0,1924.0,799.0,67391.0,1185.618258,2854.980635,311.0
Huelva (capital),21247.0,1855.0,678.0,143837.0,471.366894,1289.654261,197.0
Condado-Campiña,20645.0,1328.0,463.0,156231.0,296.356037,850.023363,149.0
Valverde del Camino,2166.0,454.0,227.0,12750.0,1780.392157,3560.784314,99.0
Nerva,708.0,210.0,89.0,5169.0,1721.803057,4062.681370,46.0
Cortegana,732.0,187.0,76.0,4602.0,1651.455889,4063.450674,24.0
Isla Cristina,3933.0,193.0,79.0,21393.0,369.279671,902.164259,23.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Rosal de la Frontera,262.0,72.0,27.0,1697.0,1591.043017,4242.781379,14.0
Cortegana,732.0,187.0,76.0,4602.0,1651.455889,4063.450674,24.0
Nerva,708.0,210.0,89.0,5169.0,1721.803057,4062.681370,46.0
Cerro de Andévalo (El),461.0,93.0,25.0,2327.0,1074.344650,3996.562097,9.0
Jabugo,306.0,87.0,35.0,2260.0,1548.672566,3849.557522,9.0
Marines (Los),44.0,15.0,8.0,399.0,2005.012531,3759.398496,3.0
Valverde del Camino,2166.0,454.0,227.0,12750.0,1780.392157,3560.784314,99.0
Valdelarco,13.0,8.0,2.0,231.0,865.800866,3463.203463,0.0
Linares de la Sierra,24.0,9.0,7.0,262.0,2671.755725,3435.114504,1.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

,Confirmados PDIA,Confirmados PDIA 14 días,Confirmados PDIA 7 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Paymogo,121.0,7.0,1.0,1162.0,86.058520,602.409639,0.0,0.142857
Alájar,71.0,13.0,2.0,759.0,263.504611,1712.779974,1.0,0.153846
Santa Bárbara de Casa,70.0,6.0,1.0,1043.0,95.877277,575.263663,0.0,0.166667
Manzanilla,261.0,22.0,4.0,2118.0,188.857413,1038.715770,0.0,0.181818
Villanueva de las Cruces,51.0,5.0,1.0,387.0,258.397933,1291.989664,1.0,0.200000
Fuenteheridos,83.0,14.0,3.0,709.0,423.131171,1974.612130,2.0,0.214286
Puerto Moral,39.0,9.0,2.0,278.0,719.424460,3237.410072,1.0,0.222222
Campillo (El),262.0,57.0,13.0,2024.0,642.292490,2816.205534,6.0,0.228070
Cumbres Mayores,301.0,35.0,8.0,1749.0,457.404231,2001.143511,2.0,0.228571
